# Fined-Grained Sentiment Analysis on Movie Reviews

In [43]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.


# Load Data

In [153]:
cols = ['productId','userId','user_name','helpfulness','score','time','summary','text']
df = pd.read_csv("movies_100k.csv",header=None,names=cols, encoding='utf-8')
# above line will be different depending on where you saved your data, and your file name
df.head()

,productId,userId,user_name,helpfulness,score,time,summary,text
0,B003AI2VGA,A141HP4LYPWMSR,"Brian E. Erland ""Rainbow Sphinx""",7/7,3.0,1182729600,"""There Is So Much Darkness Now ~ Come For The Miracle""","Synopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br /><br />Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the ""Wounds of Christ"" (stigmata) appear in her palms. She also claims to have received a message of hope for the Virgin Mary and soon a fanatical movement forms around her to fight against the evil that holds such a stranglehold on the area.<br /><br />Critique: Possessing a lifelong fascination with such esoteric matters as Catholic mysticism, miracles and the mysterious appearance of the stigmata, I was immediately attracted to the '05 DVD release `Virgin of Juarez'. The film offers a rather unique storyline blending current socio-political concerns, the constant flow of Mexican migrant workers back and forth across the U.S./Mexican border and the traditional Catholic beliefs of the Hispanic population. I must say I was quite surprised by the unexpected route taken by the plot and the means and methods by which the heavenly message unfolds.<br /><br />`Virgin of Juarez' is not a film that you would care to watch over and over again, but it was interesting enough to merit at least one viewing. Minnie Driver delivers a solid performance and Ana Claudia Talancon is perfect as the fragile and innocent visionary Mariela. Also starring Esai Morales and Angus Macfadyen (Braveheart)."
1,B003AI2VGA,A328S9RN3U5M68,Grady Harp,4/4,3.0,1181952000,Worthwhile and Important Story Hampered by Poor Script and Production,"THE VIRGIN OF JUAREZ is based on true events surrounding the crime problems of Juarez, Mexico reflected in the gringo exploitation of businesses in neighboring El Paso, Texas. The story contains many important facts that desperately need to be brought into the light, but the impact of the film falters because of the choices made by the writer and director.<br /><br />Karina Danes (Minnie Driver) is a journalist for a Los Angeles newspaper who has flown to Juarez to investigate the multiple (in the hundreds) killings of young women. The targets for these murders seem to be young women working in the US sponsored sweatshops in Juarez who are picked up at night after work, raped, beaten and killed. Danes is convinced the Juarez police force is doing nothing and takes on the mission of exposing the tragedies, in part due to her own past issues of being too idle with similar crimes in the US. She meets Father Herrera (Esai Morales) and a community activist Patrick (Angus MacFadyen) and together they probe the police files and follow the most recent murder, discovering along the way a survivor named Mariela (Ana Claudia Talanc&oacute;n), a frightened young girl whose memory of her rape and beating is erased by her apparent vision of the Virgin Mary. A father of one of the victims, Isidro (Jorge Cervera, Jr.) nurtures Mariela and helps her to escape the hospital, placing her in a 'church' where she becomes a 'saint' to the people of Juarez who long for the crimes to end. Mariela appears to the public with the stigmata of bleeding hands and offers hope to the victims' families. Danes works hard to discover evidence that will expose the perpetrators, taking a sheet of photos of 'most wanted men' from the police office of Detective Lauro (Jacob Vargas), and works with the police and Father Herrer

In [10]:
df.score.value_counts()

5.0    27957
4.0    10507
3.0     4922
1.0     3868
2.0     2746
Name: score, dtype: int64

Drop unnecessary cols

In [154]:
df.drop(['summary','user_name','helpfulness','time'],axis=1,inplace=True)

In [70]:
df.head()

,productId,userId,score,text
0,B003AI2VGA,A141HP4LYPWMSR,3.0,"Synopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br /><br />Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the ""Wounds of Christ"" (stigmata) appear in her palms. She also claims to have received a message of hope for the Virgin Mary and soon a fanatical movement forms around her to fight against the evil that holds such a stranglehold on the area.<br /><br />Critique: Possessing a lifelong fascination with such esoteric matters as Catholic mysticism, miracles and the mysterious appearance of the stigmata, I was immediately attracted to the '05 DVD release `Virgin of Juarez'. The film offers a rather unique storyline blending current socio-political concerns, the constant flow of Mexican migrant workers back and forth across the U.S./Mexican border and the traditional Catholic beliefs of the Hispanic population. I must say I was quite surprised by the unexpected route taken by the plot and the means and methods by which the heavenly message unfolds.<br /><br />`Virgin of Juarez' is not a film that you would care to watch over and over again, but it was interesting enough to merit at least one viewing. Minnie Driver delivers a solid performance and Ana Claudia Talancon is perfect as the fragile and innocent visionary Mariela. Also starring Esai Morales and Angus Macfadyen (Braveheart)."
1,B003AI2VGA,A328S9RN3U5M68,3.0,"THE VIRGIN OF JUAREZ is based on true events surrounding the crime problems of Juarez, Mexico reflected in the gringo exploitation of businesses in neighboring El Paso, Texas. The story contains many important facts that desperately need to be brought into the light, but the impact of the film falters because of the choices made by the writer and director.<br /><br />Karina Danes (Minnie Driver) is a journalist for a Los Angeles newspaper who has flown to Juarez to investigate the multiple (in the hundreds) killings of young women. The targets for these murders seem to be young women working in the US sponsored sweatshops in Juarez who are picked up at night after work, raped, beaten and killed. Danes is convinced the Juarez police force is doing nothing and takes on the mission of exposing the tragedies, in part due to her own past issues of being too idle with similar crimes in the US. She meets Father Herrera (Esai Morales) and a community activist Patrick (Angus MacFadyen) and together they probe the police files and follow the most recent murder, discovering along the way a survivor named Mariela (Ana Claudia Talanc&oacute;n), a frightened young girl whose memory of her rape and beating is erased by her apparent vision of the Virgin Mary. A father of one of the victims, Isidro (Jorge Cervera, Jr.) nurtures Mariela and helps her to escape the hospital, placing her in a 'church' where she becomes a 'saint' to the people of Juarez who long for the crimes to end. Mariela appears to the public with the stigmata of bleeding hands and offers hope to the victims' families. Danes works hard to discover evidence that will expose the perpetrators, taking a sheet of photos of 'most wanted men' from the police office of Detective Lauro (Jacob Vargas), and works with the police and Father Herrera to resolve the tragic chain of events that continue in Juarez. Fearing for Mariela's life, they transport her to Los Angeles where mysterious events end the story.<br /><br />The squeaky, mawkish script was written by Michael Fallon and dir

# Data Clean

In [157]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
from nltk.corpus import stopwords
import re

It looks like HTML encoding has not been converted to text, and ended up in text field as ‘&amp’,’&quot’,etc. Decoding HTML to general text will be my first step of data preparation. I will use BeautifulSoup for this.


The second part of the preparation is dealing with @mention.


Even though @mention carries a certain information (which another user that the tweet mentioned), this information doesn’t add value to build sentiment analysis model.


The third part of the cleaning is dealing with URL links, same with @mention, even though it carries some information, for sentiment analysis purpose, this can be ignored.


By looking at the above entry, I can see strange patterns of characters “\xef\xbf\xbd”. After some researching, I found that these are UTF-8 BOM.
“The UTF-8 BOM is a sequence of bytes (EF BB BF) that allows the reader to identify a file as being encoded in UTF-8.”
Sometimes the text used with hashtag can provide useful information about the tweet. It might be a bit risky to get rid of all the text together with the hashtag.
So I decided to leave the text intact and just remove the ‘#’. I will do this in the process of cleaning all the non letter characters including numbers

With above five data cleaning task, I will first define data cleaning function, and then will be applied to the whole dataset. Tokenization, stemming/lemmatization, stop words will be dealt with later stage when creating matrix with either count vectorizer or Tfidf vectorizer.

In [161]:
def clean_text(text_col):
    clean_text=[]
    for i in range(0,len(text_col)):
        text=str(text_col[i])
        text=re.sub('[^a-zA-Z]',' ',text)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        text=[lemma.lemmatize(w) for w in word_tokenize(str(text).lower())]
        stops = set(stopwords.words("english"))  
        text = [w for w in word_tokenize(str(text).lower()) if not w in stops]  
        text=' '.join(text)
        clean_text.append(text)
    return clean_text

In [162]:
df['clean_text']=clean_text(df.text)

In [163]:
df.drop('text',axis=1,inplace=True)

In [165]:
df.to_csv( "After_clean_movie.csv", index=False, quoting=0)
df.head()

productId           userId  score  \
0   B003AI2VGA   A141HP4LYPWMSR  3.0     
1   B003AI2VGA   A328S9RN3U5M68  3.0     
2   B003AI2VGA   A1I7QGUDP043DG  5.0     
3   B003AI2VGA   A1M5405JH9THP9  3.0     
4   B003AI2VGA   ATXL536YX71TR   3.0     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

# 3.CNN

I select 80% of 100000 data as train set and the other 20% as test

In [131]:
train_text=df.clean_text.values
target=df.score.values
y=to_categorical(target)
train_text = np.array(train_text)
target= np.array(target)
y = np.array(y)
print(train_text.shape,target.shape,y.shape)

(100000,) (100000,) (100000, 6)


In [212]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(80000,) (80000, 6)
(20000,) (20000, 6)


As we can see, the numbers of unique_word is 97351, the max of length of one text is 4636, which means our matrix is set

In [109]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

97351

In [110]:
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

4636

In [119]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=6

In [135]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)

In [136]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
print(X_train.shape,X_val.shape)

(80000, 4636) (20000, 4636)


Now, let’s define a simple CNN going through bigrams on a text.

In [140]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 4636, 100)         9735100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 4636, 100)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4636, 64)          19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 774       
Total para

In [141]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 80000 samples, validate on 20000 samples
Epoch 1/3
80000/80000 [==============================] - 2406s 30ms/step - loss: 1.0632 - acc: 0.5895 - val_loss: 0.9112 - val_acc: 0.6329
Epoch 2/3
80000/80000 [==============================] - 2378s 30ms/step - loss: 0.8266 - acc: 0.6674 - val_loss: 0.8433 - val_acc: 0.6604
Epoch 3/3
80000/80000 [==============================] - 3223s 40ms/step - loss: 0.6926 - acc: 0.7252 - val_loss: 0.8321 - val_acc: 0.6689
CPU times: user 6h 16min 35s, sys: 42min 23s, total: 6h 58min 59s
Wall time: 2h 13min 26s


The best validation accuracy is from the word vectors updated through training, at epoch 3 with the validation accuracy of 72.52%. 
The result is quiet not bad, because without the help of re-trained---Word2Vec, it 's really hard to get a high validation accuracy.
Maybe in the future, we are able to access fine-grained word2Vec which could work for Fine-Griained Sentiment Analysis.

# 2. Data visualisation

In order for me to implement a couple of data visualisation in the next step, I need term frequency data. What kind of words are used in the text, and how many times it is used in entire corpus. I used count vectorizer to calculate the term frequencies, even though the count vectorizer is also for fit, train and predict, but at this stage, I will just be extracting the term frequencies for the visualisation.
There are parameter options available for count vectorizer, such as removing stop words, limiting the maximum number of terms. However, in order to get a full picture of the dataset first, I implemented with stop words included, and not limiting the maximum number of terms.

There are parameter options available for count vectorizer, such as removing stop words, limiting the maximum number of terms. However, in order to get a full picture of the dataset first, I implemented with stop words included, and not limiting the maximum number of terms.

In [142]:
from sklearn.feature_extraction.text import CountVectorizer

In [143]:
cvec = CountVectorizer()

In [166]:
cvec.fit(df.clean_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Yes! All the batching and computation I did below can be done in much less lines of code (not exactly one line though). Once you transform the data with the fitted count vectorizer, you can directly get the term frequency from the sparse matrix. And all this can be done in less than a second! If you read below, you can see I spent almost 40mins to get the term frequency table. I guess I learned this lesson the hardest way possible. The code for getting the term frequency directly from sparse matrix is as below.

In [175]:
neg_doc_matrix = cvec.transform(df[df.score == 1].clean_text)
somewhat_neg_doc_matrix = cvec.transform(df[df.score == 2].clean_text)
neu_doc_matrix = cvec.transform(df[df.score == 3].clean_text)
somewhat_pos_doc_matrix = cvec.transform(df[df.score == 4].clean_text)
pos_doc_matrix = cvec.transform(df[df.score == 5].clean_text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
somewhat_neg_tf = np.sum(somewhat_neg_doc_matrix,axis=0)
neu_tf = np.sum(neu_doc_matrix,axis=0)
somewhat_pos_tf = np.sum(somewhat_pos_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
somewhat_neg = np.squeeze(np.asarray(somewhat_neg_tf))
neu = np.squeeze(np.asarray(neu_tf))
somewhat_pos = np.squeeze(np.asarray(somewhat_neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,somewhat_neg,neu,somewhat_pos,pos],columns=cvec.get_feature_names()).transpose()

OK, the term frequency data frame has been created! And as you can see the most frequent words are all stop words like “to”, “the”, etc. You might wonder why go through all the heavy processing without removing the stop words and not limiting the maximum terms, to get term frequency dominated by stop words? It is because I really want to test Zipf’s law with the result I got from the above. I need the stop words! You will see what I mean.

In order to come up with a meaningful metric which can characterise important tokens in each class, I borrowed a metric presented by Jason Kessler in PyData 2017 Seattle. In the talk, he presented a Python library called Scattertext. Even though I did not make use of the library, the metrics used in the Scattertext as a way of visualising text data are very useful in filtering meaningful tokens from the frequency data.
Intuitively, if a word appears more often in one class compared to another, this can be a good measure of how much the word is meaningful to characterise the class. In the below code I named it as 'pos_rate', and as you can see from the calculation of the code, this is defined as $${pos\_rate} = \frac{positive\ frequency} {positive\ frequency + negative\ frequency}$$

In [176]:
term_freq_df.columns = ['negative','somewhat_negative','neutral','somewhat_positive', 'positive']
term_freq_df['total'] = term_freq_df['negative'] +term_freq_df['somewhat_negative'] +term_freq_df['neutral']+term_freq_df['somewhat_positive']+term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:10]

,negative,somewhat_negative,neutral,somewhat_positive,positive,total
the,67927,68805,131872,68805,455653,793062
and,31075,30512,58133,30512,243296,393528
of,27821,26984,51778,26984,193542,327109
to,29353,27330,49237,27330,174041,307291
is,20601,19386,38363,19386,153495,251231
it,23876,20831,39594,20831,145667,250799
in,17305,16856,33286,16856,119399,203702
br,19916,18075,32910,18075,111516,200492
this,22014,14919,24843,14919,117586,194281
that,16165,14888,26952,14888,89216,162109


In order to come up with a meaningful metric which can characterise important tokens in each class, I borrowed a metric presented by Jason Kessler in PyData 2017 Seattle. In the talk, he presented a Python library called Scattertext. Even though I did not make use of the library, the metrics used in the Scattertext as a way of visualising text data are very useful in filtering meaningful tokens from the frequency data.
Intuitively, if a word appears more often in one class compared to another, this can be a good measure of how much the word is meaningful to characterise the class. In the below code I named it as 'pos_rate', and as you can see from the calculation of the code, this is defined as $${pos\_rate} = \frac{positive\ frequency} {positive\ frequency + negative\ frequency}$$

Words with highest pos_rate have zero frequency in the negative tweets, but overall frequency of these words are too low to think of it as a guideline for positive tweets.
Another metric is the frequency a words occurs in the class. This is defined as $${pos\_freq\_pct} = \frac {positive\ frequency} {\Sigma positive\ frequency}$$

In [179]:
term_freq_df['pos_freq_pct'] = term_freq_df['positive'] * 1./term_freq_df['positive'].sum()
term_freq_df.sort_values(by='pos_freq_pct', ascending=False).iloc[:10]

,negative,somewhat_negative,neutral,somewhat_positive,positive,total,pos_rate,pos_freq_pct
the,67927,68805,131872,68805,455653,793062,0.574549,0.061115
and,31075,30512,58133,30512,243296,393528,0.618243,0.032632
of,27821,26984,51778,26984,193542,327109,0.591674,0.025959
to,29353,27330,49237,27330,174041,307291,0.566372,0.023344
is,20601,19386,38363,19386,153495,251231,0.610972,0.020588
it,23876,20831,39594,20831,145667,250799,0.580812,0.019538
in,17305,16856,33286,16856,119399,203702,0.586145,0.016015
this,22014,14919,24843,14919,117586,194281,0.605237,0.015771
br,19916,18075,32910,18075,111516,200492,0.556212,0.014957
that,16165,14888,26952,14888,89216,162109,0.550346,0.011966


But since pos_freq_pct is just the frequency scaled over the total sum of the frequency, the rank of pos_freq_pct is exactly same as just the positive frequency.
What we can do now is to combine pos_rate, pos_freq_pct together to come up with a metric which reflects both pos_rate and pos_freq_pct. Even though both of these can take a value ranging from 0 to 1, pos_rate has much wider range actually spanning from 0 to 1, while all the pos_freq_pct values are squashed within the range smaller than 0.015. If we average these two numbers, pos_rate will be too dominant, and will not reflect both metrics effectively.
So here we use harmonic mean instead of arithmetic mean. "Since the harmonic mean of a list of numbers tends strongly toward the least elements of the list, it tends (compared to the arithmetic mean) to mitigate the impact of large outliers and aggravate the impact of small ones." The harmonic mean H of the positive real number x1,x2,...xn is defined as $${H} = \frac {n}{\sum_{i=1}^{n}\ \frac{1}{x_i}}$$

In [181]:
from scipy.stats import hmean

term_freq_df['pos_hmean'] = term_freq_df.apply(lambda x: (hmean([x['pos_rate'], x['pos_freq_pct']])
                                                                   if x['pos_rate'] > 0 and x['pos_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df.sort_values(by='pos_hmean', ascending=False).iloc[:10]

,negative,somewhat_negative,neutral,somewhat_positive,positive,total,pos_rate,pos_freq_pct,pos_rate_normcdf,pos_freq_pct_normcdf,pos_hmean
the,67927,68805,131872,68805,455653,793062,0.574549,0.061115,0.448159,1.0,0.110479
and,31075,30512,58133,30512,243296,393528,0.618243,0.032632,0.493582,1.0,0.061993
of,27821,26984,51778,26984,193542,327109,0.591674,0.025959,0.465914,1.0,0.049736
to,29353,27330,49237,27330,174041,307291,0.566372,0.023344,0.439715,1.0,0.044839
is,20601,19386,38363,19386,153495,251231,0.610972,0.020588,0.486001,1.0,0.039833
it,23876,20831,39594,20831,145667,250799,0.580812,0.019538,0.454642,1.0,0.037804
in,17305,16856,33286,16856,119399,203702,0.586145,0.016015,0.460173,1.0,0.031177
this,22014,14919,24843,14919,117586,194281,0.605237,0.015771,0.480026,1.0,0.030742
br,19916,18075,32910,18075,111516,200492,0.556212,0.014957,0.429262,1.0,0.029131
that,16165,14888,26952,14888,89216,162109,0.550346,0.011966,0.423249,1.0,0.023423


In [206]:
term_freq_df.sort_values(by='pos_hmean', ascending=False).iloc[140:150]

,negative,somewhat_negative,neutral,somewhat_positive,positive,total,pos_rate,pos_freq_pct,pos_rate_normcdf,pos_freq_pct_normcdf,pos_hmean,neg_rate,neg_freq_pct,neg_hmean,neg_rate_normcdf,neg_freq_pct_normcdf
while,948,1095,2368,1095,6891,12397,0.555860,0.000924,0.428901,0.999218,0.001845,0.076470,0.000807,0.001597,0.476384,0.997712
man,808,815,1686,815,6818,10942,0.623104,0.000914,0.498651,0.999123,0.001826,0.073844,0.000688,0.001363,0.471728,0.992063
where,1285,1159,2211,1159,6784,12598,0.538498,0.000910,0.411160,0.999074,0.001817,0.102000,0.001094,0.002164,0.521724,0.999942
watching,1240,941,1504,941,6723,11349,0.592387,0.000902,0.466655,0.998982,0.001801,0.109261,0.001055,0.002091,0.534593,0.999900
every,878,638,1030,638,6707,9891,0.678091,0.000900,0.555817,0.998956,0.001797,0.088768,0.000747,0.001482,0.498222,0.995650
video,1149,729,1301,729,6646,10554,0.629714,0.000891,0.505545,0.998852,0.001780,0.108869,0.000978,0.001939,0.533899,0.999713
look,1505,1255,2236,1255,6638,12889,0.515013,0.000890,0.387455,0.998838,0.001778,0.116766,0.001281,0.002534,0.547857,0.999997
version,1295,881,1724,881,6603,11384,0.580025,0.000886,0.453826,0.998774,0.001769,0.113756,0.001102,0.002183,0.542543,0.999949
performance,465,659,1627,659,6531,9941,0.656976,0.000876,0.533937,0.998631,0.001750,0.046776,0.000396,0.000785,0.424072,0.915261
actor,1201,1113,1762,1113,6504,11693,0.556230,0.000872,0.429281,0.998574,0.001742,0.102711,0.001022,0.002024,0.522985,0.999841


 The same for the other 4 sentiment

In [185]:
term_freq_df['neg_rate'] = term_freq_df['negative'] * 1./term_freq_df['total']
term_freq_df['neg_freq_pct'] = term_freq_df['negative'] * 1./term_freq_df['negative'].sum()
term_freq_df['neg_hmean'] = term_freq_df.apply(lambda x: (hmean([x['neg_rate'], x['neg_freq_pct']])
                                                                   if x['neg_rate'] > 0 and x['neg_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df['neg_rate_normcdf'] = normcdf(term_freq_df['neg_rate'])
term_freq_df['neg_freq_pct_normcdf'] = normcdf(term_freq_df['neg_freq_pct'])
term_freq_df.sort_values(by='neg_freq_pct', ascending=False).iloc[:10]

,negative,somewhat_negative,neutral,somewhat_positive,positive,total,pos_rate,pos_freq_pct,pos_rate_normcdf,pos_freq_pct_normcdf,pos_hmean,neg_rate,neg_freq_pct,neg_hmean,neg_rate_normcdf,neg_freq_pct_normcdf
the,67927,68805,131872,68805,455653,793062,0.574549,0.061115,0.448159,1.0,0.110479,0.085652,0.057816,0.069033,0.492686,1.0
and,31075,30512,58133,30512,243296,393528,0.618243,0.032632,0.493582,1.0,0.061993,0.078965,0.026449,0.039626,0.480812,1.0
to,29353,27330,49237,27330,174041,307291,0.566372,0.023344,0.439715,1.0,0.044839,0.095522,0.024984,0.039608,0.510222,1.0
of,27821,26984,51778,26984,193542,327109,0.591674,0.025959,0.465914,1.0,0.049736,0.085051,0.023680,0.037045,0.491619,1.0
it,23876,20831,39594,20831,145667,250799,0.580812,0.019538,0.454642,1.0,0.037804,0.095200,0.020322,0.033494,0.509650,1.0
this,22014,14919,24843,14919,117586,194281,0.605237,0.015771,0.480026,1.0,0.030742,0.113310,0.018737,0.032157,0.541755,1.0
is,20601,19386,38363,19386,153495,251231,0.610972,0.020588,0.486001,1.0,0.039833,0.082000,0.017535,0.028891,0.486200,1.0
br,19916,18075,32910,18075,111516,200492,0.556212,0.014957,0.429262,1.0,0.029131,0.099336,0.016951,0.028961,0.516995,1.0
in,17305,16856,33286,16856,119399,203702,0.586145,0.016015,0.460173,1.0,0.031177,0.084953,0.014729,0.025105,0.491444,1.0
that,16165,14888,26952,14888,89216,162109,0.550346,0.011966,0.423249,1.0,0.023423,0.099717,0.013759,0.024181,0.517672,1.0


In [197]:
term_freq_df.sort_values(by='neg_freq_pct', ascending=False).iloc[110:120]

,negative,somewhat_negative,neutral,somewhat_positive,positive,total,pos_rate,pos_freq_pct,pos_rate_normcdf,pos_freq_pct_normcdf,pos_hmean,neg_rate,neg_freq_pct,neg_hmean,neg_rate_normcdf,neg_freq_pct_normcdf
look,1505,1255,2236,1255,6638,12889,0.515013,0.000890,0.387455,0.998838,0.001778,0.116766,0.001281,0.002534,0.547857,0.999997
should,1493,1143,1764,1143,5661,11204,0.505266,0.000759,0.377733,0.995235,0.001516,0.133256,0.001271,0.002518,0.576793,0.999996
too,1483,1824,3478,1824,8018,16627,0.482228,0.001075,0.355070,0.999886,0.002146,0.089192,0.001262,0.002489,0.498977,0.999996
most,1430,1569,2972,1569,11491,19031,0.603804,0.001541,0.478534,1.000000,0.003075,0.075141,0.001217,0.002395,0.474027,0.999991
worst,1411,368,370,368,457,2974,0.153665,0.000061,0.109233,0.571275,0.000123,0.474445,0.001201,0.002396,0.956676,0.999989
ve,1397,1010,1670,1010,7378,12465,0.591897,0.000990,0.466146,0.999649,0.001976,0.112074,0.001189,0.002353,0.539570,0.999986
quot,1396,1291,3184,1291,15075,22237,0.677924,0.002022,0.555644,1.000000,0.004032,0.062778,0.001188,0.002332,0.452157,0.999986
over,1382,1323,2318,1323,10422,16768,0.621541,0.001398,0.497022,0.999999,0.002789,0.082419,0.001176,0.002319,0.486943,0.999983
nothing,1346,939,1346,939,2917,7487,0.389609,0.000391,0.269675,0.906612,0.000782,0.179778,0.001146,0.002277,0.655755,0.999973
him,1346,1412,2957,1412,10334,17461,0.591833,0.001386,0.466079,0.999999,0.002766,0.077086,0.001146,0.002258,0.477477,0.999973


In [139]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

In [137]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         9735100   
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 9,789,954
Trainable params: 9,789,954
Non-trainable params: 0
_________________________________________________________________
